## Measuring model ddG accuracy on protein variants
The goal of this notebook is to experiment with model ddG prediction capabilities, and exploring the possibility of predicting ddG of variants that were not supported previously.

In [2]:
#imports
import sys
sys.path.append('..')    # add parent directory to path
import numpy as np
import matplotlib.pyplot as plt
from model.hydro_net import PEM
from model.model_cfg import CFG
from Utils.train_utils import *
from Utils.pdb_parser import get_pdb_data
import torch
import pandas as pd
import os
import glob

Using device: cpu


In [3]:
#load model
# Import the model
model = PEM(layers=CFG.num_layers,gaussian_coef=CFG.gaussian_coef).to(CFG.device)
# Get total number of parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {total_params}")
# Upload model weights
CFG.model_path = '../data/Trained_models/'
epoch = 25
model_dict = torch.load(CFG.model_path+f"{epoch}_final_model.pt",map_location=CFG.device,weights_only=False)
model.load_state_dict(model_dict['model_state_dict'])

Number of parameters: 393517


<All keys matched successfully>

### Experimenting with a single protein

In [4]:
#Configuration
remove_indels = True
one_mute = True
# Constants
COORDS = 'coords_tensor.pt'
DELTA_G = 'deltaG.pt'
MASKS = 'mask_tensor.pt'
ONE_HOT = 'one_hot_encodings.pt'
PROTT5_EMBEDDINGS = 'prott5_embeddings'
VAL_RATIO = 0.2
RANDOM_SEED = 42
NANO_TO_ANGSTROM = 0.1
TM_PATH = './data/Processed_K50_dG_datasets/TM_proteins.csv'
DEBUG = False

In [5]:
# Function from AllProteinValidationDataset
def load_embedding_tensor(embeddings_dir):
    embeddings = []
    all_embedding_files = sorted(glob.glob(os.path.join(embeddings_dir, 'prott5_embedding_*.pt')),
                                 key=lambda x: int(os.path.splitext(x)[0].split('_')[-1]))

    for filename in all_embedding_files:
        if filename.endswith('.pt'):
            embedding_tensor = torch.load(filename, map_location=torch.device('cpu'))  # Ensure loading on CPU
            embeddings.append(embedding_tensor)

    return torch.vstack(embeddings)

In [6]:
#Data preparation
protein_path = r"mutation_data\1A0N"
mutations_path = r"mutation_data\1A0N.csv"

mutations = pd.read_csv(mutations_path)

if remove_indels:
    mutations = mutations[~mutations['mut_type'].str.contains('ins|del')].reset_index(drop=True)
    
# Load and preprocess the data for each protein
coords_tensor = torch.load(os.path.join(protein_path, COORDS))
delta_g_tensor = torch.load(os.path.join(protein_path, DELTA_G))
mask_tensor = torch.load(os.path.join(protein_path, MASKS))
one_hot_tensor = torch.load(os.path.join(protein_path, ONE_HOT))
embedding_tensor = load_embedding_tensor(os.path.join(protein_path, PROTT5_EMBEDDINGS))

# remove the mutations with more than one mutation
if one_mute:
    one_mut_index = mutations[~mutations['mut_type'].str.contains(':')]
    mutations = mutations.loc[one_mut_index.index]
    delta_g_tensor = delta_g_tensor[one_mut_index.index]
    one_hot_tensor = one_hot_tensor[one_mut_index.index]
    embedding_tensor = embedding_tensor[one_mut_index.index]
    
mutations_data = {
    'name': protein_path,
    'mutations': mutations['mut_type'].to_list(),
    'prott5': embedding_tensor,
    'coords': coords_tensor,
    'one_hot': one_hot_tensor,
    'delta_g': delta_g_tensor,
    'masks': mask_tensor
}


C:\Users\darea\AppData\Local\Temp\ipykernel_448\1661907473.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  coords_tensor = torch.load(os.path.join(protein_path, COORDS)

In [7]:
#print mutation data information
print(f"Protein: {mutations_data['name']}")
print(f"Number of mutations: {len(mutations_data['mutations'])}")
print(f"Protein T5 embeddings shape: {mutations_data['prott5'].shape}")
print(f"Protein coords shape: {mutations_data['coords'].shape}")
print(f"Protein one hot encodings shape: {mutations_data['one_hot'].shape}")
print(f"Protein delta G shape: {mutations_data['delta_g'].shape}")
print(f"Protein masks shape: {mutations_data['masks'].shape}")


Protein: mutation_data\1A0N
Number of mutations: 2210
Protein T5 embeddings shape: torch.Size([2210, 58, 1024])
Protein coords shape: torch.Size([58, 4, 3])
Protein one hot encodings shape: torch.Size([2210, 58, 21])
Protein delta G shape: torch.Size([2210])
Protein masks shape: torch.Size([58])


In [8]:
#get wt folded graph
#assuming wt is the first sequence
seq_one_hot = mutations_data['one_hot'][0]
proT5_emb = mutations_data['prott5'][0]
mask = mutations_data['masks'][0]
coords_tensor = mutations_data['coords']
protein_graph = get_graph( coords_tensor ,seq_one_hot, proT5_emb,mask)
print(protein_graph.shape)
#get wt unfolded graph
protein_unfolded_graph = get_unfolded_graph( coords_tensor ,seq_one_hot, proT5_emb,mask)
#calculate dG
with torch.no_grad():
    Gf = model(protein_graph.unsqueeze(0))
    Gf = Gf.cpu().numpy()
    Gf = Gf[0]
    print(Gf.shape) 
    print(f"The energy of the folded protein is: {Gf}")

    Gu = model(protein_unfolded_graph.unsqueeze(0))
    Gu = Gu.cpu().numpy()
    Gu = Gu[0]
    print(Gu.shape)
    print(f"The energy of the unfolded protein unfolded structure is: {Gu}")
    
# Calculate the deltaG
pred_deltaG = Gu - Gf
print(pred_deltaG)


torch.Size([58, 1093])
()
The energy of the folded protein is: -18.69211196899414
()
The energy of the unfolded protein unfolded structure is: -17.820093154907227
0.8720188


In [10]:
#Calculate the dG for first 50 mutations
pred_dG = []
for i in range(1,51):
    seq_one_hot = mutations_data['one_hot'][i]
    proT5_emb = mutations_data['prott5'][i]
    mask = mutations_data['masks'][0]
    coords_tensor = mutations_data['coords']
    protein_graph = get_graph( coords_tensor ,seq_one_hot, proT5_emb,mask)
    protein_unfolded_graph = get_unfolded_graph( coords_tensor ,seq_one_hot, proT5_emb,mask)
    with torch.no_grad():
        Gf = model(protein_graph.unsqueeze(0))
        Gf = Gf.cpu().numpy()
        Gf = Gf[0]
        Gu = model(protein_unfolded_graph.unsqueeze(0))
        Gu = Gu.cpu().numpy()
        Gu = Gu[0]
        deltaG = Gu - Gf
        pred_dG.append(deltaG)

print(pred_dG)
#average dG
print(np.mean(pred_dG))
#avarage ddG
print(np.mean(pred_dG)-deltaG)
#calculate the correlation
true_dG = mutations_data['delta_g'][1:51]
print(true_dG)
print(np.corrcoef(pred_dG,true_dG))


[3.1495705, -1.6930447, 1.9707108, 1.2984486, -0.60608673, 3.23205, 1.3747749, 0.42900276, 1.7317505, -1.9769535, 0.579792, -2.7454567, -0.41097832, 1.1421661, -0.9649792, 0.9585037, -2.9456139, 0.95069885, 0.24905014, -2.2694016, 2.1236248, -2.1156101, 0.9643059, -0.019481659, 0.11757088, -1.1760864, -2.9799347, 0.7286129, 0.36824036, -1.2744942, -0.07924652, -0.6281853, -0.20812225, 0.16975212, -0.6832409, 1.3318291, 1.019146, -1.4051647, -3.0741062, -0.41801453, 1.4475613, 1.0226927, 0.017400742, 1.2712479, 3.5291262, -0.867218, 0.616663, -0.18375015, 0.7868595, -0.34464836]
0.07022667
0.41487503
tensor([2.5827, 2.5673, 2.5520, 2.5951, 2.6898, 2.6362, 2.6432, 2.6558, 2.5567,
        2.3979, 2.4710, 2.6645, 2.5505, 2.5875, 2.6857, 2.7244, 2.7452, 2.6042,
        2.8106, 2.8407, 2.8857, 2.4839, 2.6115, 2.3851, 2.3999, 2.4302, 2.3272,
        2.5744, 2.4337, 2.2445, 2.3058, 2.4119, 2.6847, 2.8259, 2.6960, 2.7206,
        2.5569, 2.6176, 2.4842, 2.3598, 1.5899, 1.8028, 1.0520, 1.9977, 1